## Train DGMR

In [ ]:
import torch.utils.data.dataset
from dgmr import DGMR
from datasets import load_dataset
from torch.utils.data import DataLoader

from pytorch_lightning.callbacks import ModelCheckpoint
import wandb

wandb.init(project="dgmr")
from numpy.random import default_rng
import os
import numpy as np
from pathlib import Path
import tensorflow as tf
from pytorch_lightning import Callback, Trainer
from pytorch_lightning.loggers import  WandbLogger
from pytorch_lightning.utilities import rank_zero_only
    
from src.dataio import DGMRDataModule

In [ ]:
wandb_logger = WandbLogger(logger="dgmr")
model_checkpoint = ModelCheckpoint(
    monitor="train/g_loss",
    dirpath="./",
    filename="best",
)

trainer = Trainer(
    max_epochs=1000,
    logger=wandb_logger,
    callbacks=[model_checkpoint],
    gpus=1,
    precision=32,
    # accelerator="tpu", devices=8
)

In [2]:
# model = DGMR()
model = DGMR.from_pretrained("openclimatefix/dgmr")
sampler = Sampler.from_pretrained("openclimatefix/dgmr-sampler")
discriminator = Discriminator.from_pretrained("openclimatefix/dgmr-discriminator")
latent_stack = LatentConditioningStack.from_pretrained("openclimatefix/dgmr-latent-conditioning-stack")
context_stack = ContextConditioningStack.from_pretrained("openclimatefix/dgmr-context-conditioning-stack")
generator = Generator(conditioning_stack=context_stack, latent_stack=latent_stack, sampler=sampler)

In [ ]:
datamodule = DGMRDataModule(batch_size=1)
trainer.fit(model, datamodule)